# 📝 Exercícios - Orquestração de Estado com LangGraph

## Performance – IA para Devs | Desenvolvimento e Orquestração de Agentes de IA para o Setor Financeiro

---

### 📋 Instruções Gerais

Este notebook contém **3 exercícios práticos** baseados nos conceitos aprendidos na Aula 3:

| Exercício | Nível | Tópico |
|-----------|-------|--------|
| 1 | ⭐ Básico | Grafos Lineares e Estado |
| 2 | ⭐⭐ Intermediário | Arestas Condicionais |
| 3 | ⭐⭐⭐ Avançado | Human-in-the-Loop e Persistência |

**Dicas:**
- Leia atentamente o enunciado e os requisitos de cada exercício
- Use o notebook da Aula 3 como referência
- Teste seu código com os casos de teste fornecidos
- Documente seu código com comentários explicativos

---

## 🔧 Configuração do Ambiente

Antes de começar os exercícios, configure o ambiente:

```bash
pip install langgraph langchain langchain-openai langchain-core
pip install pydantic python-dotenv
```

Configure também suas variáveis de ambiente (chave da OpenAI) usando um arquivo `.env` ou diretamente no código.

---

## ⭐ Exercício 1: Pipeline de Processamento de Pedidos

### Contexto

Você foi contratado para criar um sistema de processamento de pedidos de um e-commerce. O fluxo é **linear** e deve passar pelas seguintes etapas:

```
START → Validar Pedido → Calcular Frete → Aplicar Desconto → Finalizar → END
```

### Requisitos

1. **Defina o Estado** (`EstadoPedido`) com os seguintes campos:
   - `pedido_id`: str
   - `cliente`: str
   - `itens`: list[dict] (cada item com "nome", "preco", "quantidade")
   - `subtotal`: float
   - `frete`: float
   - `desconto`: float
   - `total`: float
   - `status`: str
   - `historico`: Annotated[list[str], operator.add]

2. **Implemente os Nós:**
   - `validar_pedido`: Valida se há itens no pedido e calcula o subtotal
   - `calcular_frete`: Frete = R$ 15 se subtotal < R$ 100, senão frete grátis
   - `aplicar_desconto`: 10% de desconto se subtotal >= R$ 200
   - `finalizar_pedido`: Calcula o total (subtotal + frete - desconto)

3. **Monte o Grafo** e compile

4. **Teste** com o pedido fornecido

### Caso de Teste

```python
pedido_teste = {
    "pedido_id": "PED-001",
    "cliente": "Maria Silva",
    "itens": [
        {"nome": "Camiseta", "preco": 79.90, "quantidade": 2},
        {"nome": "Calça Jeans", "preco": 159.90, "quantidade": 1}
    ],
    "subtotal": 0.0,
    "frete": 0.0,
    "desconto": 0.0,
    "total": 0.0,
    "status": "NOVO",
    "historico": ["Pedido criado"]
}
```

**Resultado Esperado:**
- Subtotal: 79.90×2 + 159.90 = **R$ 319.70**
- Frete: **R$ 0** (grátis, pois subtotal > 100)
- Desconto: **R$ 31.97** (10% de 319.70, pois subtotal > 200)
- Total: 319.70 + 0 - 31.97 = **R$ 287.73**

---

---

## ⭐⭐ Exercício 2: Sistema de Triagem de Tickets de Suporte

### Contexto

Você precisa criar um sistema de triagem automática de tickets de suporte técnico. O sistema deve classificar os tickets por prioridade e direcioná-los para a equipe correta.

### Fluxo do Sistema

```
                          ┌─────────────────┐
                          │     START       │
                          └────────┬────────┘
                                   │
                                   ▼
                          ┌─────────────────┐
                          │ Analisar Ticket │
                          └────────┬────────┘
                                   │
                      ┌────────────┼────────────┐
                      │            │            │
                      ▼            ▼            ▼
              ┌───────────┐ ┌───────────┐ ┌───────────┐
              │   ALTA    │ │   MÉDIA   │ │   BAIXA   │
              │ (Urgente) │ │ (Normal)  │ │ (FAQ)     │
              └─────┬─────┘ └─────┬─────┘ └─────┬─────┘
                    │             │             │
                    ▼             ▼             ▼
              ┌───────────┐ ┌───────────┐ ┌───────────┐
              │Escalar p/ │ │ Fila de   │ │ Resposta  │
              │ Gerente   │ │ Atendim.  │ │ Automática│
              └─────┬─────┘ └─────┬─────┘ └─────┬─────┘
                    │             │             │
                    └─────────────┼─────────────┘
                                  ▼
                          ┌─────────────────┐
                          │  Registrar Log  │
                          └────────┬────────┘
                                   │
                                   ▼
                          ┌─────────────────┐
                          │      END        │
                          └─────────────────┘
```

### Requisitos

1. **Defina o Estado** (`EstadoTicket`) com:
   - `ticket_id`: str
   - `cliente`: str
   - `assunto`: str
   - `descricao`: str
   - `palavras_chave`: list[str]
   - `prioridade`: str ("ALTA", "MÉDIA", "BAIXA")
   - `destino`: str
   - `resposta`: str
   - `historico`: Annotated[list[str], operator.add]

2. **Regras de Classificação** (`analisar_ticket`):
   - ALTA: palavras-chave contêm "urgente", "sistema fora", "crítico", "parado"
   - MÉDIA: palavras-chave contêm "erro", "bug", "problema", "não funciona"
   - BAIXA: demais casos ou palavras como "dúvida", "como fazer", "tutorial"

3. **Implemente os Nós:**
   - `analisar_ticket`: Classifica a prioridade
   - `escalar_gerente`: Define destino = "Gerente de Plantão"
   - `fila_atendimento`: Define destino = "Fila de Suporte N2"
   - `resposta_automatica`: Gera resposta automática com FAQ
   - `registrar_log`: Registra o ticket no histórico

4. **Função de Roteamento** (`rotear_por_prioridade`):
   - Retorna o nome do próximo nó baseado na prioridade

5. **Teste** com os 3 tickets fornecidos

### Casos de Teste

**Ticket 1: Alta prioridade**
```python
ticket_urgente = {
    "ticket_id": "TKT-001",
    "cliente": "Empresa XYZ",
    "assunto": "Sistema de pagamentos fora do ar",
    "descricao": "Não conseguimos processar nenhum pagamento desde às 10h",
    "palavras_chave": ["sistema fora", "urgente", "pagamentos"],
    "prioridade": "",
    "destino": "",
    "resposta": "",
    "historico": ["Ticket aberto"]
}
# Esperado: ALTA → Gerente de Plantão
```

**Ticket 2: Média prioridade**
```python
ticket_erro = {
    "ticket_id": "TKT-002",
    "cliente": "João Santos",
    "assunto": "Erro ao gerar relatório",
    "descricao": "Quando clico em gerar relatório mensal, aparece uma mensagem de erro",
    "palavras_chave": ["erro", "relatório"],
    "prioridade": "",
    "destino": "",
    "resposta": "",
    "historico": ["Ticket aberto"]
}
# Esperado: MÉDIA → Fila de Suporte N2
```

**Ticket 3: Baixa prioridade**
```python
ticket_duvida = {
    "ticket_id": "TKT-003",
    "cliente": "Ana Costa",
    "assunto": "Como fazer para exportar dados?",
    "descricao": "Gostaria de saber como exportar meus dados para Excel",
    "palavras_chave": ["como fazer", "dúvida", "exportar"],
    "prioridade": "",
    "destino": "",
    "resposta": "",
    "historico": ["Ticket aberto"]
}
# Esperado: BAIXA → Resposta Automática
```

---

---

## ⭐⭐⭐ Exercício 3: Sistema de Aprovação de Férias com Human-in-the-Loop

### Contexto

Você precisa criar um sistema de aprovação de solicitações de férias para uma empresa. O sistema deve:
- Aprovar automaticamente solicitações simples (até 5 dias, sem conflito)
- Exigir aprovação do gestor para casos mais complexos
- Persistir o estado para que aprovações possam ser feitas posteriormente

### Fluxo do Sistema

```
START → Validar Solicitação → Verificar Conflitos → [Decisão]
                                                        │
                    ┌───────────────────────────────────┼───────────────────────────────────┐
                    │                                   │                                   │
              SEM CONFLITO                        COM CONFLITO                         INVÁLIDO
              (≤ 5 dias)                          (ou > 5 dias)                              │
                    │                                   │                                   │
                    ▼                                   ▼                                   ▼
            ┌───────────────┐                 ┌───────────────┐                    ┌───────────────┐
            │   Aprovar     │                 │   Aguardar    │                    │   Rejeitar    │
            │  Automático   │                 │   Aprovação   │                    │   Automático  │
            └───────┬───────┘                 │    (HITL)     │                    └───────┬───────┘
                    │                         └───────┬───────┘                            │
                    │                                 │                                    │
                    │                    ┌────────────┴────────────┐                       │
                    │                    │                         │                       │
                    │               APROVADO                   REJEITADO                   │
                    │                    │                         │                       │
                    │                    ▼                         ▼                       │
                    │            ┌───────────────┐         ┌───────────────┐               │
                    │            │  Confirmar    │         │  Notificar    │               │
                    │            │   Férias      │         │   Rejeição    │               │
                    │            └───────┬───────┘         └───────┬───────┘               │
                    │                    │                         │                       │
                    └────────────────────┴─────────────────────────┴───────────────────────┘
                                                        │
                                                        ▼
                                                      END
```

### Requisitos

1. **Defina o Estado** (`EstadoFerias`) com:
   - `solicitacao_id`: str
   - `funcionario`: dict (nome, departamento, dias_disponiveis)
   - `periodo`: dict (data_inicio, data_fim, dias_solicitados)
   - `status`: str (PENDENTE, AGUARDANDO_APROVACAO, APROVADO, REJEITADO)
   - `conflitos`: list[str]
   - `requer_aprovacao_gestor`: bool
   - `aprovacao_gestor`: Optional[dict] (aprovador, decisao, data, comentario)
   - `motivo_rejeicao`: str
   - `historico`: Annotated[list[str], operator.add]

2. **Implemente os Nós:**
   - `validar_solicitacao`: Verifica se tem dias suficientes
   - `verificar_conflitos`: Simula verificação de conflitos com outros funcionários
   - `aprovar_automatico`: Aprova solicitações simples (≤ 5 dias, sem conflito)
   - `aguardar_aprovacao`: Pausa para Human-in-the-Loop
   - `processar_aprovacao_gestor`: Processa decisão do gestor
   - `confirmar_ferias`: Confirma e registra férias
   - `notificar_rejeicao`: Notifica rejeição

3. **Use MemorySaver** para persistência de estado

4. **Teste os cenários:**
   - Aprovação automática (3 dias, sem conflito)
   - Aprovação pelo gestor (10 dias)
   - Rejeição automática (dias insuficientes)

### Casos de Teste

**Caso 1: Aprovação Automática** (3 dias, sem conflito)
```python
solicitacao_simples = {
    "solicitacao_id": "FER-001",
    "funcionario": {
        "nome": "Carlos Silva",
        "departamento": "Marketing",
        "dias_disponiveis": 30
    },
    "periodo": {
        "data_inicio": "2025-03-10",
        "data_fim": "2025-03-12",
        "dias_solicitados": 3
    },
    "status": "PENDENTE",
    "conflitos": [],
    "requer_aprovacao_gestor": False,
    "aprovacao_gestor": None,
    "motivo_rejeicao": "",
    "historico": ["Solicitação criada"]
}
# Esperado: Status = APROVADO (automático)
```

**Caso 2: Aprovação com HITL** (10 dias, requer gestor)
```python
solicitacao_longa = {
    "solicitacao_id": "FER-002",
    "funcionario": {
        "nome": "Maria Souza",
        "departamento": "TI",
        "dias_disponiveis": 30
    },
    "periodo": {
        "data_inicio": "2025-07-01",
        "data_fim": "2025-07-10",
        "dias_solicitados": 10
    },
    "status": "PENDENTE",
    "conflitos": [],
    "requer_aprovacao_gestor": False,
    "aprovacao_gestor": None,
    "motivo_rejeicao": "",
    "historico": ["Solicitação criada"]
}
# Esperado: Status = AGUARDANDO_APROVACAO (pausa para gestor)
# Após aprovação do gestor: Status = APROVADO
```

**Caso 3: Rejeição Automática** (dias insuficientes)
```python
solicitacao_invalida = {
    "solicitacao_id": "FER-003",
    "funcionario": {
        "nome": "Pedro Costa",
        "departamento": "RH",
        "dias_disponiveis": 5  # Apenas 5 dias disponíveis
    },
    "periodo": {
        "data_inicio": "2025-08-01",
        "data_fim": "2025-08-15",
        "dias_solicitados": 15  # Solicitando 15 dias
    },
    "status": "PENDENTE",
    "conflitos": [],
    "requer_aprovacao_gestor": False,
    "aprovacao_gestor": None,
    "motivo_rejeicao": "",
    "historico": ["Solicitação criada"]
}
# Esperado: Status = REJEITADO, Motivo = Dias insuficientes
```

---


## 💡 Dicas Extras

1. **Para visualizar seu grafo:**
```python
from IPython.display import Image, display
display(Image(seu_grafo.get_graph().draw_mermaid_png()))
```

2. **Para debug do estado:**
```python
def imprimir_estado(state, titulo):
    print(f"\n📊 {titulo}")
    for k, v in state.items():
        print(f"   {k}: {v}")
```

3. **Para HITL com interrupt:**
```python
motor = workflow.compile(
    checkpointer=MemorySaver(),
    interrupt_before=["aguardar_aprovacao"]
)
```

4. **Consulte os notebooks anteriores:**
   - `02_Multiplas_entradas.ipynb` - Conceitos de estado e reducer
   - `03_LangGraph_Orquestracao.ipynb` - Grafos, arestas condicionais
   - `04_MultiAgentes_MCP.ipynb` - HITL e persistência

---

## ✅ Entrega

- Crie um novo notebook com suas soluções
- Certifique-se de que todas as células executam sem erros
- Adicione comentários explicando sua lógica
- Visualize cada grafo com `draw_mermaid_png()`

**Bom trabalho!** 🚀